In [1]:
import sys
sys.path.append('../../')

In [2]:
# importnb allows us to load the code from other notebooks
from importnb import Notebook
with Notebook():
    import OfficeAutomation.common as common  # will probably become OfficeAutomation.common
    import OfficeAutomation.lamps as lamps
    import OfficeAutomation.electrical as electrical

In [3]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [5]:
class SplitterTest(common.TestClass):
        
    def test_connect_devices_influence_req_electricity(self):
        class Container(crest.Entity):
            state = current = crest.State()
            lamp1 = lamps.Lamp()
            lamp2 = lamps.Lamp()
            splitter = electrical.Splitter()
            
            def __init__(self):
                self.splitter.connect_devices([self.lamp1, self.lamp2])
        
        system = Container()
        for lamp in [system.lamp1, system.lamp2]:
            lamp.need_watt.value = 100
            lamp.switch_in.value = "on"
            lamp.current = lamp.on
        sim = crestsim.Simulator(system)
        sim.stabilise()
        
        self.assertEqual(system.lamp1.req_electricity_out.value, 100)
        self.assertEqual(system.lamp2.req_electricity_out.value, 100)
        self.assertEqual(system.splitter.req_electricity_out.value, 200)
        
    def test_connect_devices_sets_electricity_of_devices(self):
        class Container(crest.Entity):
            state = current = crest.State()
            lamp1 = lamps.Lamp()
            lamp2 = lamps.Lamp()
            splitter = electrical.Splitter()
            
            def __init__(self):
                self.splitter.connect_devices([self.lamp1, self.lamp2])
        
        system = Container()
        for lamp in [system.lamp1, system.lamp2]:
            lamp.need_watt.value = 100
            lamp.switch_in.value = "on"
            lamp.current = lamp.on
            
        system.splitter.electricity_in.value = 300
        
        sim = crestsim.Simulator(system)
        sim.stabilise()
        
        self.assertEqual(system.splitter.elec_lamp1.value, 100)
        self.assertEqual(system.splitter.elec_lamp2.value, 100)
        self.assertEqual(system.lamp1.electricity_in.value, 100)
        self.assertEqual(system.lamp2.electricity_in.value, 100)
        
    def test_connect_devices_not_enough_electricity_sets_electricity_of_devices_zero(self):
        class Container(crest.Entity):
            state = current = crest.State()
            lamp1 = lamps.Lamp()
            lamp2 = lamps.Lamp()
            splitter = electrical.Splitter()
            
            def __init__(self):
                self.splitter.connect_devices([self.lamp1, self.lamp2])
        
        system = Container()
        for lamp in [system.lamp1, system.lamp2]:
            lamp.need_watt.value = 100
            lamp.switch_in.value = "on"
            lamp.current = lamp.on
            
        system.splitter.electricity_in.value = 100
        
        sim = crestsim.Simulator(system)
        sim.stabilise()
        
        self.assertEqual(system.splitter.elec_lamp1.value, 0)
        self.assertEqual(system.splitter.elec_lamp2.value, 0)
        self.assertEqual(system.lamp1.electricity_in.value, 0)
        self.assertEqual(system.lamp2.electricity_in.value, 0)
        
# SplitterTest.runall()